### Example for NaCl cell optimization using ASE-LBFGS and Conquest

In [1]:
import os

from ase.build import bulk
from ase.calculators.conquest import Conquest
from ase.constraints import UnitCellFilter
from ase.optimize import LBFGS
from ase.io.conquest import Conquest_orthorhombic_check

#### Define Conquest environment

In [2]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-release-f-rework-output-ase/src/Conquest'
#os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-develop/src/Conquest'
os.environ['CQ_PP_PATH'] = '/Users/lioneltruflandier/Conquest-develop-outdated/pseudo-and-pao'
os.environ['CQ_GEN_BASIS_CMD'] = '/Users/lioneltruflandier/CONQUEST-release-develop/tools/BasisGeneration/MakeIonFiles'

#### Directory for storing calculation files

In [3]:
working_directory = 'cq_example_rocksalt_optcell_ase'

#### Generate rocksalt struture with $a=6.0$ Ang.

In [4]:
rocksalt = bulk('NaCl', crystalstructure='rocksalt', a=6.0)

# Conquest can only handle orthorhombic cells
rocksalt = Conquest_orthorhombic_check(rocksalt,verbose=True)

## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase/ase/io/conquest.py , line: 108
>>>>  Current cell not orthorhombic:
Bravais lattice:
   FCC(a=6)
Space group:
   Fm-3m (225)
Cell parameters (Ang. and degree):
   a =       4.242641
   b =       4.242641
   c =       4.242641
   alpha =  60.0000
   beta  =  60.0000
   gamma =  60.0000
Cartesian atomic positions (Ang.)
  Na      0.000000      0.000000      0.000000
  Cl      3.000000      0.000000      0.000000


## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase/ase/io/conquest.py , line: 122
>>>>  New orthorhombic cell:
Bravais lattice:
   CUB(a=6)
Space group:
   Fm-3m (225)
Cell parameters (Ang. and degree):
   a =       6.000000
   b =       6.000000
   c =       6.000000
   alpha =  90.0000
   beta  =  90.0000
   gamma =  90.0000
Cartesian atomic positions (Ang.)
  Na      0.000000      0.000000      0.000000
  Cl      3.000000      0.000000      0.000000
  Na      0.000000      3.000000      

#### Setup Conquest atomic basis set

In [5]:
basis = {'Na' : {'gen_basis'            : True,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann'},
         'Cl' : {'gen_basis'            : False,
                 'basis_size'           : 'medium',
                 'pseudopotential_type' : 'hamann'
                 }}

#basis = {'Na' : {'file' : 'Na_PBE_DZP_CQ.ion'},
#         'Cl' : {'file' : 'Cl_PBE_DZP_CQ.ion'}}
#
#basis = {'Na' : {'gen_basis'            : False,
#                 'basis_size'           : 'medium',
#                 'pseudopotential_type' : 'hamann',
#                 'xc'                   : 'PBE' },
#         'Cl' : {'file' : 'Cl_PBE_DZP_CQ.ion', 'xc' : 'PBE'}
#         }

#### Setup calculation using Conquest as calculator

In [6]:
conquest_flags = {'IO.WriteOutToASEFile': True}

calc = Conquest(directory=working_directory,
                grid_cutoff=80,
                xc='PBE',
                self_consistent=True,
                basis=basis,
                kpts=[3,3,3],
                nspin=1,
                **conquest_flags)

input basis:
Na {'gen_basis': True, 'basis_size': 'medium', 'pseudopotential_type': 'hamann'}
input basis:
Cl {'gen_basis': False, 'basis_size': 'medium', 'pseudopotential_type': 'hamann'}


#### Run calculation

In [7]:
rocksalt.calc = calc
dft_energy    = rocksalt.get_potential_energy()

## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase/ase/calculators/conquest.py , line: 275
>>>>  Na.ion taken from cq_example_rocksalt_optcell_ase/Na.ion
## ConquestWarning ##
file: /Users/lioneltruflandier/Conquest-ase/ase/ase/calculators/conquest.py , line: 275
>>>>  Cl.ion taken from cq_example_rocksalt_optcell_ase/Cl.ion
output basis:
Na {'gen_basis': True, 'basis_size': 'medium', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}
output basis:
Cl {'gen_basis': False, 'basis_size': 'medium', 'pseudopotential_type': 'hamann', 'xc': 'PBE'}


In [8]:
ucf = UnitCellFilter(rocksalt)
opt = LBFGS(ucf)

In [9]:
opt.run(fmax=0.005)

       Step     Time          Energy         fmax
LBFGS:    0 17:46:53    -6772.649997        0.9917
LBFGS:    1 17:47:17    -6772.677717        0.9754
LBFGS:    2 17:47:44    -6773.017788        0.6573
LBFGS:    3 17:48:12    -6773.209043        0.2989
LBFGS:    4 17:48:42    -6773.233950        0.1865
LBFGS:    5 17:49:14    -6773.242080        0.0198
LBFGS:    6 17:49:44    -6773.242243        0.0024


True

#### Compute final lattice constant

In [10]:
exp_lattice = 5.64
thr_lattice = (8*rocksalt.get_volume()/len(rocksalt))**(1.0/3.0)
print('exp lattice contant = {:8.4f}'.format(exp_lattice))
print('th  lattice contant = {:8.4f}'.format(thr_lattice))

exp lattice contant =   5.6400
th  lattice contant =   5.6077
